In [1]:
import ontology.Schema as sc
import spacy
import qai.QAI_Manager as q
import os
import pickle

In [2]:
ontology_path = "input/medibot/ontology.ttl"
schema = sc.getGraph(ontology_path)
classes_index = sc.load_classes_index(schema)
properties_index = sc.load_properties_index(schema)

In [3]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [4]:
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [5]:
random_state(1234)

In [6]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [7]:
inputs = ['what is the highest price for the drug buscopan?']

In [8]:
paraphrases = set()

In [9]:
for input in inputs:
    print("-"*100)
    print("Input_phrase: ", input)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase=input,max_return_phrases = 10)
    paraphrases.update(para_phrases)
    for para_phrase in para_phrases:
        print(para_phrase)

----------------------------------------------------------------------------------------------------
Input_phrase:  what is the highest price for the drug buscopan?
----------------------------------------------------------------------------------------------------
('tell me the best price for buscopan?', 30)
('tell me the highest price for buscopan?', 26)
("what's the highest price of buscopan?", 25)
("what's the biggest price for a buscopan?", 24)


In [10]:
for input in inputs:
    print("-"*100)
    print("Input_phrase: ", input)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase=input,do_diverse=True,max_return_phrases = 10)
    paraphrases.update(para_phrases)
    for para_phrase in para_phrases:
        print(para_phrase)

----------------------------------------------------------------------------------------------------
Input_phrase:  what is the highest price for the drug buscopan?
----------------------------------------------------------------------------------------------------
('tell me the highest price for buscopan?', 26)
("what's the highest price of buscopan?", 25)
("what's the highest price for buscopan?", 23)
('what is the highest price for buscopan?', 21)


In [11]:
paraphrases

{('tell me the best price for buscopan?', 30),
 ('tell me the highest price for buscopan?', 26),
 ('what is the highest price for buscopan?', 21),
 ("what's the biggest price for a buscopan?", 24),
 ("what's the highest price for buscopan?", 23),
 ("what's the highest price of buscopan?", 25)}

In [12]:
from googletrans import Translator

In [13]:
translator = Translator()

In [14]:
paraphrases_pt= []

In [15]:
paraphrases_l= [paraprhase[0] for paraprhase in paraphrases]

In [16]:
for paraphrase in paraphrases_l:
    paraphrase_pt = translator.translate(paraphrase, dest='pt')
    paraphrases_pt.append(str(paraphrase_pt.text))
    print(f'{paraphrase_pt.origin}->{paraphrase_pt.text}')

tell me the highest price for buscopan?->Diga -me o preço mais alto para Buscopan?
what's the biggest price for a buscopan?->Qual é o maior preço para um Buscopan?
what's the highest price for buscopan?->Qual é o preço mais alto para Buscopan?
what is the highest price for buscopan?->Qual é o preço mais alto para Buscopan?
tell me the best price for buscopan?->Diga -me o melhor preço para Buscopan?
what's the highest price of buscopan?->Qual é o preço mais alto de Buscopan?


In [17]:
paraphrases_pt

['Diga -me o preço mais alto para Buscopan?',
 'Qual é o maior preço para um Buscopan?',
 'Qual é o preço mais alto para Buscopan?',
 'Qual é o preço mais alto para Buscopan?',
 'Diga -me o melhor preço para Buscopan?',
 'Qual é o preço mais alto de Buscopan?']

In [18]:
from nltk.corpus import wordnet as wn

In [19]:
wn.synsets("drugs",pos=wn.NOUN)

[Synset('drug.n.01')]

In [20]:
synset = wn.synset('drug.n.01')

In [21]:
synset.definition()

'a substance that is used as a medicine or narcotic'

In [22]:
synset.hypernyms()

[Synset('agent.n.03')]

In [23]:
synset.hyponyms()

[Synset('abortifacient.n.01'),
 Synset('agonist.n.04'),
 Synset('anesthetic.n.01'),
 Synset('antagonist.n.03'),
 Synset('anti-tnf_compound.n.01'),
 Synset('antisyphilitic.n.01'),
 Synset('arsenical.n.01'),
 Synset('botanical.n.01'),
 Synset('brand-name_drug.n.01'),
 Synset('controlled_substance.n.01'),
 Synset('dilator.n.02'),
 Synset('diuretic_drug.n.01'),
 Synset('drug_of_abuse.n.01'),
 Synset('feosol.n.01'),
 Synset('fergon.n.01'),
 Synset('fertility_drug.n.01'),
 Synset('generic_drug.n.01'),
 Synset('intoxicant.n.02'),
 Synset('levallorphan.n.01'),
 Synset('medicine.n.02'),
 Synset('miotic_drug.n.01'),
 Synset('mydriatic.n.01'),
 Synset('narcotic.n.01'),
 Synset('pentoxifylline.n.01'),
 Synset('psychoactive_drug.n.01'),
 Synset('psychotropic_agent.n.01'),
 Synset('relaxant.n.01'),
 Synset('soporific.n.01'),
 Synset('stimulant.n.02'),
 Synset('suppressant.n.01'),
 Synset('synergist.n.01'),
 Synset('virility_drug.n.01')]

In [24]:
synset.lemmas()

[Lemma('drug.n.01.drug')]

In [25]:
[ (syn,syn.definition()) for syn in wn.synsets("produces",pos=wn.VERB)]

[(Synset('produce.v.01'), 'bring forth or yield'),
 (Synset('produce.v.02'), 'create or manufacture a man-made product'),
 (Synset('produce.v.03'), 'cause to happen, occur or exist'),
 (Synset('produce.v.04'), 'bring out for display'),
 (Synset('grow.v.07'),
  'cultivate by growing, often involving improvements by means of agricultural techniques'),
 (Synset('produce.v.06'), 'bring onto the market or release'),
 (Synset('grow.v.08'),
  'come to have or undergo a change of (physical features and attributes)')]

In [26]:
synsetProduces = wn.synset('produce.v.02')

In [27]:
synsetProduces.lemmas()

[Lemma('produce.v.02.produce'),
 Lemma('produce.v.02.make'),
 Lemma('produce.v.02.create')]

In [28]:
paraphrases = set()
inputs = ['What Laboratory produces the drug buscopan?']
for input in inputs:
    print("-"*100)
    print("Input_phrase: ", input)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase=input,max_return_phrases = 10)
    paraphrases.update(para_phrases)
    for para_phrase in para_phrases:
        print(para_phrase)

----------------------------------------------------------------------------------------------------
Input_phrase:  What Laboratory produces the drug buscopan?
----------------------------------------------------------------------------------------------------
('tell me the company which manufactures buscopan?', 38)
('tell me the laboratory that produces buscopan?', 27)
('what laboratory produces buscopan drugs?', 27)
('what laboratory produces buscopan?', 21)
('tell me the lab that produces the drug buscopan?', 19)


In [30]:
inputs = ['what is the property for the thing something?']
paraphrases = set()
for input in inputs:
    print("-"*100)
    print("Input_phrase: ", input)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase=input,max_return_phrases = 10)
    paraphrases.update(para_phrases)
    for para_phrase in para_phrases:
        print(para_phrase)
for input in inputs:
    print("-"*100)
    print("Input_phrase: ", input)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase=input,do_diverse=True,max_return_phrases = 10)
    paraphrases.update(para_phrases)
    for para_phrase in para_phrases:
        print(para_phrase)
print("*"*100)
print("Distincts variations:")
paraphrases

----------------------------------------------------------------------------------------------------
Input_phrase:  what is the property for the thing something?
----------------------------------------------------------------------------------------------------
('what property do you have for something?', 32)
("what's the property of a thing?", 29)
('what is the property for that particular thing?', 25)
('what is a property for something?', 25)
('what is the property of something?', 24)
----------------------------------------------------------------------------------------------------
Input_phrase:  what is the property for the thing something?
----------------------------------------------------------------------------------------------------
("what's the property of the thing?", 26)
("what's the property of something?", 26)
('what is the property of something?', 24)
****************************************************************************************************
Distincts variat

{('what is a property for something?', 25),
 ('what is the property for that particular thing?', 25),
 ('what is the property of something?', 24),
 ('what property do you have for something?', 32),
 ("what's the property of a thing?", 29),
 ("what's the property of something?", 26),
 ("what's the property of the thing?", 26)}

In [38]:
paraphrases_l = []
for item in paraphrases:
    text = item[0]
    text = text.replace('property','<$Property_Label>')
    text = text.replace('thing','<$Class_Label>')
    text = text.replace('something','<$resource>')
    paraphrases_l.append(text)
    print(text)

what is the <$Property_Label> for that particular <$Class_Label>?
what's the <$Property_Label> of a <$Class_Label>?
what is a <$Property_Label> for some<$Class_Label>?
what's the <$Property_Label> of some<$Class_Label>?
what's the <$Property_Label> of the <$Class_Label>?
what <$Property_Label> do you have for some<$Class_Label>?
what is the <$Property_Label> of some<$Class_Label>?


In [55]:
paraphrases_l = [item[0] for item in paraphrases]
paraphrases_l_pt = []
for paraphrase in paraphrases_l:
    paraphrase_pt = translator.translate(paraphrase, dest='pt')
    print(f'{paraphrase_pt.origin}->{paraphrase_pt.text}')
    paraphrases_l_pt.append(paraphrase_pt.text)
for item in paraphrases_l_pt:
    text = item
    text = text.replace('propriedade','<$Property_Label>')
    text = text.replace('alguma coisa','<$resource>')
    text = text.replace('coisa','<$Class_Label>')
    text = text.replace('algo','<$resource>')
    print(text)

what is the property for that particular thing?->Qual é a propriedade para essa coisa em particular?
what's the property of a thing?->Qual é a propriedade de uma coisa?
what is a property for something?->O que é uma propriedade para alguma coisa?
what's the property of something?->Qual é a propriedade de algo?
what's the property of the thing?->Qual é a propriedade da coisa?
what property do you have for something?->Que propriedade você tem para alguma coisa?
what is the property of something?->Qual é a propriedade de algo?
Qual é a <$Property_Label> para essa <$Class_Label> em particular?
Qual é a <$Property_Label> de uma <$Class_Label>?
O que é uma <$Property_Label> para <$resource>?
Qual é a <$Property_Label> de <$resource>?
Qual é a <$Property_Label> da <$Class_Label>?
Que <$Property_Label> você tem para <$resource>?
Qual é a <$Property_Label> de <$resource>?


In [1]:
import json
import spacy

In [2]:
with open("qald_9_plus_train_dbpedia.json","r") as qald_txt:
    qald = json.load(qald_txt)

In [3]:
questions = [ question['question'][0]['string'] for question in qald['questions']]

In [4]:
questions

['List all boardgames by GMT.',
 'Who developed Skype?',
 'Which people were born in Heraklion?',
 'In which U.S. state is Area 51 located?',
 'Who is the mayor of New York City?',
 'Which countries have places with more than two caves?',
 'Where did Abraham Lincoln die?',
 'Which airports does Air China serve?',
 'Give me all actors starring in movies directed by and starring William Shatner.',
 'What are the official languages of the Philippines?',
 'Give me all Danish films.',
 'Which movies starring Brad Pitt were directed by Guy Ritchie?',
 'Give me the grandchildren of Bruce Lee.',
 'Which other weapons did the designer of the Uzi develop?',
 'Who is the owner of Universal Studios?',
 'Which state of the USA has the highest population density?',
 'Which monarchs were married to a German?',
 'Which organizations were founded in 1950?',
 'Who created the comic Captain America?',
 'Give me the Apollo 14 astronauts.',
 'Who wrote the book The pillars of the Earth?',
 'Which state of 

In [6]:
model = spacy.load("en_core_web_sm")

In [9]:
vectors = []
for sentence in questions:
    vectors.append(model(sentence.lower()).vector)

In [10]:
from sklearn.cluster import KMeans
import numpy as np


In [11]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(vectors)

In [13]:
clusters = {}
for idx,label in enumerate(kmeans.labels_):
    if not label in clusters:
        clusters[label] = []
    clusters[label].append(questions[idx])

In [14]:
clusters

{0: ['List all boardgames by GMT.',
  'Was Marc Chagall a jew?',
  'Was Margaret Thatcher a chemist?',
  'Is Christian Bale starring in Batman Begins?',
  'Is Darth Vader Luke’s father?',
  'Is Cola a beverage?',
  'Are tree frogs a type of amphibian?',
  'Is horse racing a sport?',
  'Is the wife of president Obama called Michelle?',
  'Is Frank Herbert still alive?',
  'Was the wife of president Lincoln called Mary?',
  'Did Arnold Schwarzenegger attend a university?',
  "Who was John F. Kennedy's vice president?",
  'Is Barack Obama a democrat?',
  'List the children of Margaret Thatcher.',
  'List all games by GMT.',
  'Did Kaurismäki ever win the Grand Prix at Cannes?',
  'Is Christian Bale starring in Velvet Goldmine?',
  'Was Natalie Portman born in the United States?',
  'Was U.S. president Jackson involved in a war?',
  'Did Tesla win a nobel prize in physics?',
  'Does Neymar play for Real Madrid?',
  'Do Prince Harry and Prince William have the same parents?',
  'Is proinsul

In [16]:
for group in clusters:
    print(f"Grupo {group}:")
    print(clusters[group][0:5])

Grupo 0:
['List all boardgames by GMT.', 'Was Marc Chagall a jew?', 'Was Margaret Thatcher a chemist?', 'Is Christian Bale starring in Batman Begins?', 'Is Darth Vader Luke’s father?']
Grupo 6:
['Who developed Skype?', 'Which states border Illinois?', 'Who developed Minecraft?', 'Who created Goofy?', 'Who killed John Lennon?']
Grupo 7:
['Which people were born in Heraklion?', 'Which movies starring Brad Pitt were directed by Guy Ritchie?', 'Which monarchs were married to a German?', 'Which organizations were founded in 1950?', 'Which spaceflights were launched from Baikonur?']
Grupo 4:
['In which U.S. state is Area 51 located?', 'Which airports does Air China serve?', 'Which other weapons did the designer of the Uzi develop?', 'Which U.S. states are in the same timezone as Utah?', 'Which U.S. states possess gold minerals?']
Grupo 3:
['Who is the mayor of New York City?', 'Who is the owner of Universal Studios?', 'Who created the comic Captain America?', 'Who wrote the book The pillars 

In [ ]:
python -m spacy init vectors pt-br skip_s100.zip pt_br
python -m spacy init vectors language vectors_to_load name_output_model

DB.DBA.TTLP_MT (file_to_string_output ('drugsFull.nt'), '', '<http://localhost:8890/DAV/drugs>',512);